<a href="https://colab.research.google.com/github/JeongHanJun/Colab/blob/master/tf2_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.5.0'

In [4]:
%tensorflow_version 2.x

## 1. Import Libraries

In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical# one-hot encoding
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

%matplotlib inline

print(tf.__version__)
print(keras.__version__)


2.5.0
2.5.0


## Hyper Parameters

In [45]:
learning_rate = 0.01
my_epochs = 15
my_batch_size = 100

## 3. Import MNIST data from keras

In [46]:
mnist = keras.datasets.mnist
class_name = [ str(i) for i in range(10) ]
print(class_name)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [47]:
#load data (train, test)
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

# Normalize val <= 1 
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# ndim 차원 변경
train_images = np.expand_dims(train_images, axis = -1)
test_images = np.expand_dims(test_images, axis = -1)

# label을 통한 one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

## 4. Model Functions

In [48]:
# Function Model Layer

def create_model_relu():
  inputs = keras.Input( shape = (28, 28, 1) )# keras.Input 에서 I가 소문자가 아닌 대문자 I
  
  conv1 = keras.layers.Conv2D( filters = 32, kernel_size = [3, 3], padding = "SAME", activation = tf.nn.relu )(inputs)# tf.nn.leaky_relu도 있음
  pool1 = keras.layers.MaxPool2D( padding = "SAME" )(conv1)
  
  conv2 = keras.layers.Conv2D( filters = 64, kernel_size = [3, 3], padding = "SAME", activation = tf.nn.relu )(pool1)
  pool2 = keras.layers.MaxPool2D( padding = "SAME" )(conv2)
  
  conv3 = keras.layers.Conv2D( filters = 128, kernel_size=[3, 3], padding = "SAME", activation=tf.nn.relu )(pool2)
  pool3 = keras.layers.MaxPool2D( padding = "SAME" )(conv3)

  pool3_flat = keras.layers.Flatten()(pool3)

  dense4 = keras.layers.Dense( units = 256, activation = tf.nn.relu)(pool3_flat )
  drop4 = keras.layers.Dropout( rate = 0.4 )(dense4)

  logits = keras.layers.Dense(units = 10, activation = tf.nn.softmax)(drop4)

  return keras.Model( inputs = inputs, outputs = logits )# inputs =inputs 에서 앞의 i는 소문자 i이고, parameter i 이다.  뒤의 I는 소문자 I이고 함수 상단 inputs = keras.Input으로 받은 Inputs이다.


def create_model_leaky_relu():
  inputs = keras.Input( shape = (28, 28, 1) )# keras.Input 에서 I가 소문자가 아닌 대문자 I
  
  conv1 = keras.layers.Conv2D( filters = 32, kernel_size = [3, 3], padding = "SAME", activation = tf.nn.leaky_relu )(inputs)
  pool1 = keras.layers.MaxPool2D( padding = "SAME" )(conv1)
  
  conv2 = keras.layers.Conv2D( filters = 64, kernel_size = [3, 3], padding = "SAME", activation = tf.nn.leaky_relu )(pool1)
  pool2 = keras.layers.MaxPool2D( padding = "SAME" )(conv2)
  
  conv3 = keras.layers.Conv2D( filters = 128, kernel_size=[3, 3], padding = "SAME", activation=tf.nn.leaky_relu )(pool2)
  pool3 = keras.layers.MaxPool2D( padding = "SAME" )(conv3)

  pool3_flat = keras.layers.Flatten()(pool3)

  dense4 = keras.layers.Dense( units = 256, activation = tf.nn.leaky_relu)(pool3_flat )
  drop4 = keras.layers.Dropout( rate = 0.4 )(dense4)

  logits = keras.layers.Dense(units = 10, activation = tf.nn.softmax)(drop4)

  return keras.Model( inputs = inputs, outputs = logits )# inputs = inputs 에서 앞의 i는 소문자 i이고, parameter i 이다.  뒤의 I는 소문자 I이고 함수 상단 inputs = keras.Input으로 받은 Inputs이다.

model1 = create_model_relu()
model2 = create_model_leaky_relu()


In [49]:
model1.summary

<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f12fd49fcd0>>

In [50]:
model1.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 4, 4, 128)         0   

In [51]:
model2.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 4, 4, 128)         0   

## 5. Training

In [52]:
# make CNN Model Structure and Compile
model1.compile( 
    loss = 'categorical_crossentropy', # cross entropy loss
    optimizer = 'adam', # adam optimizer
    metrics = ['accuracy']# 정확도 측정
)

# Execute Learning
model1.fit(
    train_images, train_labels, 
    batch_size = my_batch_size, # 2. Hyper Parameter 참고 , batch_size 는 조정가능 , 한번 학습할 때 얼마의 크기로 학습할 것인지
    epochs = my_epochs, # 2. Hyper Parameter 참고, epochs 도 조정가능 , 몇번 학습할 것인지
    verbose = 1, # verbose : 학습 중 출력 문구 설정( 아래 Epoch 1/15 600/600  어쩌구 나오는 이 문구들에 대한 설정 , verbose = 0 : silent , verbose = 1 : progress bar , verbose = 2 : one line per each epoch )
    validation_data = (test_images, test_labels)
)

score = model1.evaluate(test_images, test_labels, verbose = 0)# check test result
print('Test loss :', score[0] )
print('Test accuracy :', score[1])

Epoch 1/15
600/600 [==============================] - 86s 142ms/step - loss: 0.1932 - accuracy: 0.9380 - val_loss: 0.0458 - val_accuracy: 0.9839
Epoch 2/15
600/600 [==============================] - 85s 141ms/step - loss: 0.0551 - accuracy: 0.9839 - val_loss: 0.0339 - val_accuracy: 0.9894
Epoch 3/15
600/600 [==============================] - 84s 140ms/step - loss: 0.0388 - accuracy: 0.9877 - val_loss: 0.0310 - val_accuracy: 0.9908
Epoch 4/15
600/600 [==============================] - 84s 141ms/step - loss: 0.0297 - accuracy: 0.9909 - val_loss: 0.0231 - val_accuracy: 0.9927
Epoch 5/15
600/600 [==============================] - 84s 140ms/step - loss: 0.0242 - accuracy: 0.9924 - val_loss: 0.0233 - val_accuracy: 0.9923
Epoch 6/15
600/600 [==============================] - 84s 140ms/step - loss: 0.0216 - accuracy: 0.9934 - val_loss: 0.0256 - val_accuracy: 0.9911
Epoch 7/15
600/600 [==============================] - 84s 140ms/step - loss: 0.0179 - accuracy: 0.9945 - val_loss: 0.0217 - val_ac

In [52]:
# make CNN Model Structure and Compile
model2.compile( 
    loss = 'categorical_crossentropy', # cross entropy loss
    optimizer = 'adam', # adam optimizer
    metrics = ['accuracy']# 정확도 측정
)

# Execute Learning
model2.fit(
    train_images, train_labels, 
    batch_size = my_batch_size, # 2. Hyper Parameter 참고 , batch_size 는 조정가능 , 한번 학습할 때 얼마의 크기로 학습할 것인지
    epochs = my_epochs, # 2. Hyper Parameter 참고, epochs 도 조정가능 , 몇번 학습할 것인지
    verbose = 1, # verbose : 학습 중 출력 문구 설정( 아래 Epoch 1/15 600/600  어쩌구 나오는 이 문구들에 대한 설정 , verbose = 0 : silent , verbose = 1 : progress bar , verbose = 2 : one line per each epoch )
    validation_data = (test_images, test_labels)
)

score = model2.evaluate(test_images, test_labels, verbose = 0)# check test result
print('Test loss :', score[0] )
print('Test accuracy :', score[1])